In [ ]:
!pip install -U hopsworks 

In [ ]:
!pip install dataframe_image

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install --upgrade protobuf

In [ ]:
!pip install streamlit_folium

In [ ]:
!pip install pyngrok

In [ ]:
!pip install python-dotenv

In [48]:
ngrok.set_auth_token("2Jzz5UrcRRGTHRB1KeKe4M38bwG_mDhqRzgG61co9WY6dMVG")

INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.8/dist-packages/pyngrok/bin/ngrok
2023-01-07 13:06:02.724 Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.8/dist-packages/pyngrok/bin/ngrok


In [47]:
from pyngrok import ngrok

In [84]:
%%writefile app.py
import streamlit as st
import hopsworks
import os
import joblib
import pandas as pd
import dataframe_image as dfi
import streamlit as st
import joblib
import numpy as np
import folium
from streamlit_folium import st_folium, folium_static
import json
import time
from datetime import timedelta, datetime
from branca.element import Figure

#----------------------------------HEADER + TITLE
def fancy_header(text, font_size=24):
    res = f'<span style="color:#ff5f27; font-size: {font_size}px;">{text}</span>'
    st.markdown(res, unsafe_allow_html=True )
st.title('⛅️Air Quality Prediction Project🌩')

progress_bar = st.sidebar.header('⚙️ Working Progress')
progress_bar = st.sidebar.progress(0)
st.write(36 * "-")

#-----------------------------------CONNECTING TO HOPSWORKS 
fancy_header('\n📡 Connecting to Hopsworks Feature Store...')

project = hopsworks.login()

st.write("Successfully connected!✔️")
progress_bar.progress(20)
#------------------------------------  PREDICTION

st.write(36 * "-")
fancy_header('\n☁️ Getting data for predict ...')
city = 'Paris'
date = ''

WEATHER_API_KEY = os.getenv('WEATHER_API_KEY')
df_weather = get_weather_df(city, date, WEATHER_API_KEY)
next_7_days = df_weather["date"]
data_7_days = df_weather.drop('date', axis=1)

mr = project.get_model_registry()
model = mr.get_best_model("aqi_model", "r2", "max")
model_dir = model.download()
model = joblib.load(model_dir + "/aqi_model.pkl")

pred_7_days = model.predict(data_7_days)

df = pd.DataFrame(data=pred_7_days, index=next_7_days, columns=[f"AQI Predictions for the next 7 days"], dtype=int)

# >>>cols_names_dict = {"old name": "new name"}
# >>>df = df.rename(columns=cols_names_dict)

progress_bar.progress(60)
#----------------------------------

st.write(36 * "-")
fancy_header(f"🗺 Processing the map...")

fig = Figure(width=550,height=350)

my_map = folium.Map(location=[58, 20], zoom_start=3.71)
fig.add_child(my_map)
folium.TileLayer('Stamen Terrain').add_to(my_map)
folium.TileLayer('Stamen Toner').add_to(my_map)
folium.TileLayer('Stamen Water Color').add_to(my_map)
folium.TileLayer('cartodbpositron').add_to(my_map)
folium.TileLayer('cartodbdark_matter').add_to(my_map)
folium.LayerControl().add_to(my_map)

#Determining Paris
coords = [48.862, 2.346]
#-----
city = "Pris"
country="France"

#-----
text=""
for column in df.columns:
        text += f"""
                    <tr>
                        <th>{column}:</th>
                        <td>{df.loc[0][column]}</td>
                    </tr>"""
text += """</table>  </h5>"""
#-----

folium.Marker(
        location=coords, popup=text, tooltip=f"<strong>{city}</strong>"
    ).add_to(my_map)


# call to render Folium map in Streamlit
folium_static(my_map)
progress_bar.progress(80)
st.sidebar.write("-" * 36)


st.sidebar.write(df)
progress_bar.progress(100)
st.button("Re-run")


Overwriting app.py


In [52]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'


INFO:pyngrok.ngrok:Opening tunnel named: http-80-43126356-51d1-4085-b5e2-0afca0881f59
2023-01-07 13:09:17.708 Opening tunnel named: http-80-43126356-51d1-4085-b5e2-0afca0881f59
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:17+0000 lvl=info msg="no configuration paths supplied"
2023-01-07 13:09:17.781 t=2023-01-07T13:09:17+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:17+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2023-01-07 13:09:17.793 t=2023-01-07T13:09:17+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:17+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2023-01-07 13:09:17.806 t=2023-01-07T13:09:17+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:17+0000 lvl=info msg="starting web service" obj=web addr

NgrokTunnel: "http://d063-34-71-151-161.ngrok.io" -> "http://localhost:80"


2023-01-07 13:09:18.103 t=2023-01-07T13:09:18+0000 lvl=info msg="started tunnel" obj=tunnels name="http-80-43126356-51d1-4085-b5e2-0afca0881f59 (http)" addr=http://localhost:80 url=http://d063-34-71-151-161.ngrok.io
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:18+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-43126356-51d1-4085-b5e2-0afca0881f59 addr=http://localhost:80 url=https://d063-34-71-151-161.ngrok.io
2023-01-07 13:09:18.116 t=2023-01-07T13:09:18+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-43126356-51d1-4085-b5e2-0afca0881f59 addr=http://localhost:80 url=https://d063-34-71-151-161.ngrok.io
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:18+0000 lvl=info msg=end pg=/api/tunnels id=c0fe2b467a52e07f status=201 dur=84.022204ms
2023-01-07 13:09:18.127 t=2023-01-07T13:09:18+0000 lvl=info msg=end pg=/api/tunnels id=c0fe2b467a52e07f status=201 dur=84.022204ms
INFO:pyngrok.process.ngrok:t=2023-01-07T13:09:18+0000 lvl=info msg=start pg="/api/tunnels/http-80-43